In [4]:
import pickle
import dill

In [19]:
import pandas as pd

In [110]:
df = pd.read_csv("data.csv", index_col='Datetime', parse_dates=True)

In [126]:
df[df.index >= "2018-08-02 00:00:00"]

,PJME_MW,isFuture
Datetime,,
2018-08-02 00:00:00,37158.0,False
2018-08-02 01:00:00,34283.0,False
2018-08-02 02:00:00,32094.0,False
2018-08-02 03:00:00,30543.0,False
2018-08-02 04:00:00,29791.0,False
2018-08-02 05:00:00,29854.0,False
2018-08-02 06:00:00,31197.0,False
2018-08-02 07:00:00,33182.0,False
2018-08-02 08:00:00,35645.0,False


In [28]:
df.shape

(145366, 1)

In [111]:
df.sort_index(inplace=True)

In [112]:
def create_features(df):
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype('int')
    df['moving_avg_7'] = df['PJME_MW'].rolling(window=7).mean()
    df['moving_avg_14'] = df['PJME_MW'].rolling(window=14).mean()
    df['moving_avg_30'] = df['PJME_MW'].rolling(window=30).mean()
    df['lag_1'] = df['PJME_MW'].shift(1)
    df['lag_2'] = df['PJME_MW'].shift(2)
    #df.dropna(inplace=True)

    return df

In [118]:
def add_lags(df, lag: pd.Timedelta=pd.Timedelta('1 hour')):
    target_map = df['PJME_MW'].to_dict()
    df['lag_3'] = (df.index - lag).map(target_map)
    return df

In [114]:
future = pd.date_range(start ='2018-08-03', end='2018-08-04', freq='H')
future_df = pd.DataFrame(index=future)
future_df["isFuture"] = True
df["isFuture"] = False
df_and_future = pd.concat([df, future_df])

In [123]:
add_lags(df = df_and_future,lag=pd.Timedelta('1 day')).sort_index().query('isFuture')

,PJME_MW,isFuture,lag_3
2018-08-03 00:00:00,NaN,True,37158.0
2018-08-03 01:00:00,NaN,True,34283.0
2018-08-03 02:00:00,NaN,True,32094.0
2018-08-03 03:00:00,NaN,True,30543.0
2018-08-03 04:00:00,NaN,True,29791.0
2018-08-03 05:00:00,NaN,True,29854.0
2018-08-03 06:00:00,NaN,True,31197.0
2018-08-03 07:00:00,NaN,True,33182.0
2018-08-03 08:00:00,NaN,True,35645.0
2018-08-03 09:00:00,NaN,True,37810.0


In [101]:
df_and_future = create_features(df_and_future.sort_index())
df_and_future = add_lags(df_and_future.sort_index())

In [108]:
df_and_future.sort_index()

,PJME_MW,isFuture,lag_3,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,moving_avg_7,moving_avg_14,moving_avg_30,lag_1,lag_2
2002-01-01 01:00:00,30393.0,False,NaN,1,1,1,1,2002,1,1,1,NaN,NaN,NaN,NaN,NaN
2002-01-01 02:00:00,29265.0,False,30393.0,2,1,1,1,2002,1,1,1,NaN,NaN,NaN,30393.0,NaN
2002-01-01 03:00:00,28357.0,False,29265.0,3,1,1,1,2002,1,1,1,NaN,NaN,NaN,29265.0,30393.0
2002-01-01 04:00:00,27899.0,False,28357.0,4,1,1,1,2002,1,1,1,NaN,NaN,NaN,28357.0,29265.0
2002-01-01 05:00:00,28057.0,False,27899.0,5,1,1,1,2002,1,1,1,NaN,NaN,NaN,27899.0,28357.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-03 20:00:00,NaN,True,NaN,20,4,3,8,2018,215,3,31,NaN,NaN,NaN,NaN,NaN
2018-08-03 21:00:00,NaN,True,NaN,21,4,3,8,2018,215,3,31,NaN,NaN,NaN,NaN,NaN
2018-08-03 22:00:00,NaN,True,NaN,22,4,3,8,2018,215,3,31,NaN,NaN,NaN,NaN,NaN
2018-08-03 23:00:00,NaN,True,NaN,23,4,3,8,2018,215,3,31,NaN,NaN,NaN,NaN,NaN


In [109]:
add_lags(df)[["PJME_MW", 'lag_3']]

TypeError: unsupported operand type(s) for -: 'DatetimeArray' and 'str'